In [10]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
pd.set_option("display.max_rows", 2000, "display.max_columns", None)



def get_submission(pred, Name):
    res = pd.DataFrame(pred, index = test[test.columns[0]], columns=["target"]) 
    res.to_csv(str(Name)+ ".csv")

In [2]:
train = pd.read_csv(r"Train.csv")
test = pd.read_csv(r"Test.csv")

In [3]:
# train.isna().sum()

In [4]:
train.corr()["target"]

target                                                 1.000000
target_min                                             0.818803
target_max                                             0.630801
target_variance                                        0.184818
target_count                                           0.094135
precipitable_water_entire_atmosphere                  -0.031329
relative_humidity_2m_above_ground                     -0.084835
specific_humidity_2m_above_ground                     -0.035670
temperature_2m_above_ground                           -0.038818
u_component_of_wind_10m_above_ground                  -0.148202
v_component_of_wind_10m_above_ground                  -0.055100
L3_NO2_NO2_column_number_density                       0.306772
L3_NO2_NO2_slant_column_number_density                 0.315718
L3_NO2_absorbing_aerosol_index                         0.030386
L3_NO2_cloud_fraction                                 -0.123956
L3_NO2_sensor_altitude                  

In [5]:
train_X = train.drop(train.columns[3:8], axis = 1)
train_y = train["target"]

In [6]:
# #column transformers were completely unable to work with the data for some reason
# cols = train_X.columns[3:-7]
# colTrans = make_column_transformer((SimpleImputer(), cols), (StandardScaler(), cols))
# colTrans2 = ColumnTransformer(transformers=[("impute", SimpleImputer(strategy="median"), cols), ("scale", StandardScaler(), cols)])
def DataPrep(data):
    d = data.drop(['L3_CH4_CH4_column_volume_mixing_ratio_dry_air', 'L3_CH4_aerosol_height', 'L3_CH4_aerosol_optical_depth', 'L3_CH4_sensor_azimuth_angle', 'L3_CH4_sensor_zenith_angle', 'L3_CH4_solar_azimuth_angle', 'L3_CH4_solar_zenith_angle'], axis =1)
    train_X1 = pd.DataFrame(SimpleImputer().fit_transform(d[d.columns[3:]]))
    train_X1 = pd.DataFrame(StandardScaler().fit_transform(train_X1[train_X1.columns[3:]]))
    return train_X1

In [7]:
train_X = DataPrep(train_X)
test_t = DataPrep(test)

In [8]:
train_X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
0,0.984205,0.583280,-0.498085,-0.225515,-0.072243,-0.233326,-0.932990,0.363649,1.287287,0.297965,0.680700,-1.178827,1.933392,-1.062255,-0.605883,-0.673000,0.357894,-1.214130,1.247598,0.151910,0.805225,-1.633919,-1.531354,-0.121752,-0.909561,0.215453,1.321316,0.214426,0.939271,-1.864128,-0.803492,-1.593879,1.382000,0.165612,0.968131,-1.896011,-0.295651,-1.575873,0.000000,8.126793e-16,-1.242893,0.000000,0.000000,0.000000,1.239702,0.113291,0.855125,-1.873476,2.791080e-16,-0.130665,2.625133,1.242401,0.107450,0.834703,-1.852070,-0.304863,-0.530063,-0.493884,0.134489,-1.549938,1.359537,0.181300,0.986012,-1.895832
1,1.415508,1.075924,-0.483214,-0.201152,0.320307,0.019414,-0.892257,0.366077,-0.060729,1.363467,0.599077,-0.844708,1.765770,-1.062255,-0.537746,-0.801950,0.332618,-1.043839,-0.048283,1.266825,0.719781,-1.231614,-1.436701,0.201311,-1.069416,0.226830,-0.673658,1.566471,0.748590,-1.072666,1.067560,-1.119535,-0.008850,1.435419,0.873669,-1.411042,1.013984,-0.651733,-1.067501,1.220947e+00,-1.072712,-0.527942,-1.324167,1.575274,-0.044352,1.262906,0.767990,-1.426837,8.905507e-01,0.146586,2.818198,-0.050374,1.253936,0.747653,-1.413053,0.047929,0.021511,0.126948,0.796645,-1.119248,-0.015681,1.451434,0.891448,-1.405515
2,1.895489,1.716741,1.291532,-0.305415,0.067364,0.157545,-0.900698,0.368827,-1.397632,0.866762,0.453157,-0.541267,2.116004,-0.891414,-0.755775,-0.778995,0.300013,-0.978997,-1.333497,0.748105,0.566620,-0.865927,-1.572142,0.153107,-1.012338,0.229192,-1.378477,0.885227,0.689927,-0.982554,-0.245695,-0.938919,-1.388249,0.844635,0.704346,-0.970214,0.438194,-0.486662,-1.021399,1.151963e+00,-1.007090,-0.536686,-1.182046,1.372130,-1.317836,0.728041,0.611799,-1.020850,8.107091e-01,0.298113,3.036878,-1.332490,0.719479,0.592032,-1.014345,0.047675,-0.328377,0.145693,0.601757,-0.955254,-1.379578,0.860497,0.721939,-0.959825
3,1.568062,0.955379,0.382675,-0.128701,0.109839,0.538452,-0.763719,0.367501,-1.384819,-0.181893,0.514080,-0.741610,2.148384,-0.315903,-0.511970,0.066660,0.260857,-0.866683,-1.321179,-0.347014,0.630566,-1.107364,-1.518533,0.095006,-0.748150,0.225893,-1.355146,-0.343925,0.756763,-1.260731,-0.295504,-0.626070,-1.375029,-0.402626,0.775039,-1.261261,-0.089363,0.216969,-0.972141,1.103908e+00,-0.894390,-0.515508,-1.187834,1.403085,-1.305631,-0.401161,0.677010,-1.288894,5.308957e-01,0.715950,2.931452,-1.320202,-0.408879,0.657005,-1.277587,0.145838,0.705390,0.534283,1.142077,-0.671197,-1.366506,-0.387085,0.792710,-1.254082
4,0.802195,0.814234,0.977362,-0.263631,-0.208709,2.480748,-0.857311,0.366034,1.025983,-1.622672,0.586485,-0.936299,2.312804,-0.315905,-0.735967,-0.433490,0.308031,-1.105480,0.996413,-1.851667,0.706564,-1.341993,0.155189,0.073938,-0.396979,0.222103,0.425967,-1.995441,0.836412,-1.529588,-0.089269,-1.291237,1.112407,-2.116317,0.859057,-1.544102,0.637750,-1.314332,0.824168,-9.838406e-01,-1.136791,-0.603528,0.863330,-1.006151,0.990809,-1.952645,0.754511,-1.549380,1.146211e-01,2.846614,2.814809,0.991805,-1.959166,0.734224,-1.533399,0.352758,-0.096175,0.871173,3.477086,-1.275148,1.092974,-2.101218,0.876820,-1.540042


In [8]:
params1 = {"n_estimators": np.arange(100, 500, 10), "max_depth": np.arange(1,10, 2), "subsample": np.arange(0.5,1.0,0.1), "max_features": np.arange(10,60)}
# params = {"n_estimators": np.arange(100, 500, 10), "max_depth": np.arange(1,10, 2), "max_features": np.arange(0.1, 1.0, 0.1)}

rs1 = RandomizedSearchCV(GradientBoostingRegressor(random_state=123), params1, n_iter = 100, scoring = "neg_mean_squared_error", n_jobs=-1, cv=3, random_state=123)

# rs = RandomizedSearchCV(RandomForestRegressor(random_state=123), params, n_iter = 100, scoring = "neg_mean_squared_error", n_jobs=-1, cv=3, random_state=123)

rs1.fit(train_X, train_y)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...e=123, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': array([100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220,
       230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350,
       360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480,
       490]), 'max_depth': array([1, 3, 5, 7, 9...37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])},
          pre_dispatch='2*n_jobs', random_state=12

In [18]:
rs1.best_params_

{'subsample': 0.7999999999999999,
 'n_estimators': 490,
 'max_features': 19,
 'max_depth': 9}

In [12]:
est = rs1.best_estimator_
pred2 = est.predict(test_t)
get_submission(pred2, "bestGrad1")

In [ ]:
params2 = {"n_estimators": np.arange(500,1000, 10), "max_depth": np.arange(6,20, 2), "subsample": np.arange(0.6,1.0,0.1), "max_features": np.arange(18,50)}
rs2 = RandomizedSearchCV(GradientBoostingRegressor(random_state=123), params2, n_iter = 100, scoring = "neg_mean_squared_error", n_jobs=-1, cv=3, random_state=123)
rs2.fit(train_X, train_y)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [13]:
rs2.best_params_

{'subsample': 0.7999999999999999,
 'n_estimators': 610,
 'max_features': 41,
 'max_depth': 8}

In [20]:
np.sqrt(-rs2.best_score_)

33.809336661144414

In [19]:
est_best = rs2.best_estimator_
prediction = est_best.predict(test_t)
get_submission(prediction, "bestGradientBoost")